In [1]:
import polars as pl
import numpy as np
import sys, os

this_path = '/home/ibi/Documents/GitHub/diem-thpt-analysis'
sys.path.append(this_path)
os.chdir(this_path)

In [2]:
files = os.listdir('data/raw')
files

['2018.csv',
 '2021.csv',
 '2020.csv',
 '2019.csv',
 '2022.csv',
 '2023.csv',
 '2024.csv']

In [3]:
def read_and_add_year(file):
    year = file.split('_')[0][:-4]
    df = pl.read_csv(f'data/raw/{file}')
    year_col = pl.Series('Year', [year]*df.height)
    df = df.insert_column(0, year_col)

    return df

In [4]:
def process_dtype(df):
    i64_cols = ['Year', 'CityCode', 'StudentID']
    for col in i64_cols:
        df = df.with_columns(pl.col(col).cast(pl.Int64))
    for col in df.columns[3:]:
        df = df.with_columns(pl.col(col).cast(pl.Float64))
        df = df.with_columns(pl.col(col).replace(-1, np.nan))

    return df

In [5]:
dfs = [process_dtype(read_and_add_year(file)) for file in files]
df = pl.concat(dfs)
df

Year,CityCode,StudentID,Toan,NguVan,NgoaiNgu,VatLi,HoaHoc,SinhHoc,KHTN,DiaLi,LichSu,GDCD,KHXH
i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2018,1,1000033,7.6,8.25,7.0,null,null,null,null,7.25,4.25,7.75,6.42
2018,1,1000034,6.2,6.25,9.2,null,null,null,null,7.0,5.25,8.5,6.92
2018,1,1000035,6.4,7.25,6.6,null,null,null,null,5.0,3.75,7.25,5.33
2018,1,1000037,2.8,7.0,4.2,null,null,null,null,3.5,3.5,7.25,4.75
2018,1,1000038,4.0,6.5,4.8,null,null,null,null,6.25,3.25,7.0,5.5
…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024,64,64006675,8.2,8.25,7.0,9.25,8.5,6.0,7.916667,NaN,NaN,NaN,NaN
2024,64,64006673,4.8,7.0,NaN,NaN,NaN,NaN,NaN,7.25,6.5,NaN,6.875
2024,64,64006674,5.8,8.25,4.0,6.75,5.75,4.25,5.583333,NaN,NaN,NaN,NaN


In [6]:
df = df.fill_null(float('nan'))
df

Year,CityCode,StudentID,Toan,NguVan,NgoaiNgu,VatLi,HoaHoc,SinhHoc,KHTN,DiaLi,LichSu,GDCD,KHXH
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2018.0,1.0,1.000033e6,7.6,8.25,7.0,NaN,NaN,NaN,NaN,7.25,4.25,7.75,6.42
2018.0,1.0,1.000034e6,6.2,6.25,9.2,NaN,NaN,NaN,NaN,7.0,5.25,8.5,6.92
2018.0,1.0,1.000035e6,6.4,7.25,6.6,NaN,NaN,NaN,NaN,5.0,3.75,7.25,5.33
2018.0,1.0,1.000037e6,2.8,7.0,4.2,NaN,NaN,NaN,NaN,3.5,3.5,7.25,4.75
2018.0,1.0,1.000038e6,4.0,6.5,4.8,NaN,NaN,NaN,NaN,6.25,3.25,7.0,5.5
…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024.0,64.0,6.4006675e7,8.2,8.25,7.0,9.25,8.5,6.0,7.916667,NaN,NaN,NaN,NaN
2024.0,64.0,6.4006673e7,4.8,7.0,NaN,NaN,NaN,NaN,NaN,7.25,6.5,NaN,6.875
2024.0,64.0,6.4006674e7,5.8,8.25,4.0,6.75,5.75,4.25,5.583333,NaN,NaN,NaN,NaN


In [7]:
subjects = df.columns[3:]
for subject in subjects:
    df = df.filter(pl.col(subject) != 0)

df

Year,CityCode,StudentID,Toan,NguVan,NgoaiNgu,VatLi,HoaHoc,SinhHoc,KHTN,DiaLi,LichSu,GDCD,KHXH
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2018.0,1.0,1.000033e6,7.6,8.25,7.0,NaN,NaN,NaN,NaN,7.25,4.25,7.75,6.42
2018.0,1.0,1.000034e6,6.2,6.25,9.2,NaN,NaN,NaN,NaN,7.0,5.25,8.5,6.92
2018.0,1.0,1.000035e6,6.4,7.25,6.6,NaN,NaN,NaN,NaN,5.0,3.75,7.25,5.33
2018.0,1.0,1.000037e6,2.8,7.0,4.2,NaN,NaN,NaN,NaN,3.5,3.5,7.25,4.75
2018.0,1.0,1.000038e6,4.0,6.5,4.8,NaN,NaN,NaN,NaN,6.25,3.25,7.0,5.5
…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024.0,64.0,6.4006675e7,8.2,8.25,7.0,9.25,8.5,6.0,7.916667,NaN,NaN,NaN,NaN
2024.0,64.0,6.4006673e7,4.8,7.0,NaN,NaN,NaN,NaN,NaN,7.25,6.5,NaN,6.875
2024.0,64.0,6.4006674e7,5.8,8.25,4.0,6.75,5.75,4.25,5.583333,NaN,NaN,NaN,NaN


In [8]:
columns_to_check1 = ['NgoaiNgu', 'VatLi', 'HoaHoc', 'SinhHoc', 'DiaLi', 'LichSu', 'GDCD']
columns_to_check2 = ['Toan', 'NguVan']

condition1 = pl.reduce(
    lambda a, b: a & b,
    [pl.col(col).is_nan() for col in columns_to_check1]
)

condition2 = pl.reduce(
    lambda a, b: a | b,
    [pl.col(col).is_nan() for col in columns_to_check2]
)

condition = condition1 | condition2

df = df.filter(~condition)

df

Year,CityCode,StudentID,Toan,NguVan,NgoaiNgu,VatLi,HoaHoc,SinhHoc,KHTN,DiaLi,LichSu,GDCD,KHXH
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2018.0,1.0,1.000033e6,7.6,8.25,7.0,NaN,NaN,NaN,NaN,7.25,4.25,7.75,6.42
2018.0,1.0,1.000034e6,6.2,6.25,9.2,NaN,NaN,NaN,NaN,7.0,5.25,8.5,6.92
2018.0,1.0,1.000035e6,6.4,7.25,6.6,NaN,NaN,NaN,NaN,5.0,3.75,7.25,5.33
2018.0,1.0,1.000037e6,2.8,7.0,4.2,NaN,NaN,NaN,NaN,3.5,3.5,7.25,4.75
2018.0,1.0,1.000038e6,4.0,6.5,4.8,NaN,NaN,NaN,NaN,6.25,3.25,7.0,5.5
…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024.0,64.0,6.4006669e7,3.2,5.0,NaN,NaN,NaN,NaN,NaN,6.25,4.25,NaN,5.25
2024.0,64.0,6.4006675e7,8.2,8.25,7.0,9.25,8.5,6.0,7.916667,NaN,NaN,NaN,NaN
2024.0,64.0,6.4006673e7,4.8,7.0,NaN,NaN,NaN,NaN,NaN,7.25,6.5,NaN,6.875


In [9]:
df = process_dtype(df)

for col in df.columns[3:]:
    df = df.with_columns(pl.col(col).replace(np.nan, None))

df

Year,CityCode,StudentID,Toan,NguVan,NgoaiNgu,VatLi,HoaHoc,SinhHoc,KHTN,DiaLi,LichSu,GDCD,KHXH
i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2018,1,1000033,7.6,8.25,7.0,null,null,null,null,7.25,4.25,7.75,6.42
2018,1,1000034,6.2,6.25,9.2,null,null,null,null,7.0,5.25,8.5,6.92
2018,1,1000035,6.4,7.25,6.6,null,null,null,null,5.0,3.75,7.25,5.33
2018,1,1000037,2.8,7.0,4.2,null,null,null,null,3.5,3.5,7.25,4.75
2018,1,1000038,4.0,6.5,4.8,null,null,null,null,6.25,3.25,7.0,5.5
…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024,64,64006669,3.2,5.0,null,null,null,null,null,6.25,4.25,null,5.25
2024,64,64006675,8.2,8.25,7.0,9.25,8.5,6.0,7.916667,null,null,null,null
2024,64,64006673,4.8,7.0,null,null,null,null,null,7.25,6.5,null,6.875


In [14]:
%%timeit -n 1 -r 5
df.write_csv('data/preprocessed/thpt_total.csv')

2.7 s ± 320 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
